# Naked Puts Screener

### Imports

In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import math
from scipy.stats import norm
import os
import numpy as np
import pytz
import nltk
import time
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/junlo/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Helper Functions

In [2]:
def manual_etf_tickers():
    etfs = ['INTR', 'THRN', 'QD', 'EPM', 'VTSI', 'NU', 'TZOO', 'MRAM', 'TTI', 'RAVE', 'CLBT', 'VYGR', 'HLX', 'PLX', 'CPG', 'CRESY', 'WT', 'NAT', 'OSUR', 'IRS', 'ORLA', 'MFIN', 'ESRT', 'ISSC', 'BVN', 'BBVA', 'SAN', 
    'MUFG', 'KOS', 'RES', 'HL', 'VMD', 'DAIO', 'ZYME', 'FSM', 'TGB', 'LUNA', 'SNDL', 'EXPR', 'RIG', 'IVR', 'GNW', 'SWN', 'SOS', 'NOK', 'SOLO', 'OPK', 'MNKD', 'AMRN', 'VXRT', 'HL', 'SIRI', 'KODK', 'KGC', 'ACB', 'AAOI', 
    'CRON', 'ET', 'PBR', 'BB', 'CODX', 'PAA', 'INO', 'COTY', 'HYLN', 'FCEL', 'SOL', 'INFN', 'AMC', 'CPE', 'NIO', 'GNOM', 'MORT', 'CHII', 'CHIX', 'GOVZ', 'BATT', 'MOON', 'SFYX', 'LABU', 'LCID', 'UNG', 'NIO', 'PSQ', 'SOXS', 
    'F', 'LU', 'RLX', 'NXE', 'LPL', 'DNA', 'BRFS', 'CIG', 'SID', 'BTG', 'OPEN', 'ENIC', 'AUR', 'GOL', 'AMBP', 'HMY', 'TV', 'KGC', 'TKC', 'BGC', 'AEG', 'GGB', 'STGW', 'IQ', 'TEO', 'MQ', 'GNW', 'LAZR', 'PAYO', 'RKLB', 'RNW', 
    'ICL', 'PSEC', 'YMM', 'GBTG', 'SBSW', 'JBLU', 'ADT', 'SWN', 'NTCO', 'CEPU', 'MCW', 'TAL', 'KEP', 'DISH', 'GDRX', 'IHS', 'QS', 'MPW', 'PTON', 'CHPT', 'CLVT', 'JOBY', 'UA', 'AQN', 'KOS', 'LFST', 'ALIT', 'EVEX', 'UAA', 
    'GTX', 'SOFI', 'RIG', 'TCN', 'CPG', 'ESRT', 'CD', 'BHC', 'TIXT', 'PLUG', 'HGTY', 'PAGS', 'CWK', 'VLY', 'CRCT', 'PLTK', 'VIEW', 'LXP', 'AGNC', 'RITM', 'ETRN', 'PROK', 'ARCO', 'TAC', 'LVWR', 'GRFS', 'MRVI', 'VIAV', 'HOOD', 
    'PSO', 'PACB', 'GPS', 'GDS', 'NWL', 'MLCO', 'CCCS', 'DNB', 'ROIV', 'NKLA', 'LU', 'RLX', 'DNA', 'BRFS', 'LYG', 'CIG', 'SID', 'ABEV', 'BTG', 'BBD', 'DIDIY', 'MFG', 'OPEN', 'ENIC', 'GRAB', 'AUR', 'GOL', 'AMBP', 'HMY', 
    'PSNY', 'UGP', 'NMR', 'NOK', 'SAN', 'TEF', 'BTE', 'AMC', 'BB', 'SIRI', 'HL', 'KGC', 'TV', 'TKC', 'NXE', 'LPL', 'ERIC', 'WIT', 'BGC', 'AEG', 'GGB', 'STGW', 'IQ', 'BSBR', 'ITUB', 'TEO', 'MQ', 'GNW', 'LAZR', 'UWMC', 'PAYO', 
    'NWG', 'RKLB', 'RNW', 'ICL', 'PSEC', 'YMM', 'LCID', 'GBTG', 'SBSW', 'JBLU', 'ADT', 'SWN', 'NTCO', 'TME', 'CEPU', 'MCW', 'TAL', 'KEP', 'DISH', 'GDRX', 'IHS', 'UMC', 'QS', 'MPW', 'PTON', 'CHPT', 'CLVT', 'JOBY', 'UA', 'AQN', 
    'NU', 'ASX', 'BCS', 'KOS', 'MUFG', 'LFST', 'ALIT', 'EVEX', 'BBVA', 'CX', 'UAA', 'GTX', 'SOFI', 'RIG', 'HLN', 'TCN', 'CPG', 'ESRT', 'CD', 'BHC', 'TIXT', 'SMFG', 'ELP', 'VIV', 'PLUG', 'HGTY', 'PAGS', 'CWK', 'VLY', 'VOD', 
    'SNAP', 'CRCT', 'PLTK', 'VIEW', 'LXP', 'AGNC', 'TEVA', 'AMCR', 'RITM', 'ETRN', 'PROK', 'ARCO', 'TAC', 'LVWR', 'GRFS', 'MRVI', 'VIAV', 'SUZ', 'HOOD', 'PSO', 'PACB', 'GPS', 'GDS', 'RKT', 'NWL', 'MLCO', 'CCCS', 'DNB', 'NIO', 
    'DB', 'KEY', 'ROIV', 'FLWS', 'YQ', 'TURN', 'ATNF', 'DIBS', 'XXII', 'ME', 'TSVT', 'TWOU', 'DDD', 'FEAM', 'EGHT', 'NMTRQ', 'MASS', 'AKA', 'ABCL', 'ABEO', 'ABSI', 'ACTG', 'AXDX', 'ACCO', 'ARAY', 'ACRX', 'ACER', 'ACHV', 
    'ACRS', 'ACR', 'ATNM', 'ABOS', 'AFIB', 'ADMP', 'ADAP', 'ADPT', 'ADCT', 'ADEA', 'ADIL', 'ACET', 'ADTX', 'ADMA', 'ADT', 'ADTN', 'ADV', 'ADN', 'ADVM', 'AGLE', 'AEG', 'AMTX', 'AEMD', 'AEVA', 'LIDR', 'AFMD', 'MITT', 'UAVS', 
    'AGEN', 'AGTI', 'AGNC', 'API', 'AGRI', 'AIRG', 'MIMO', 'AKBA', 'KERN', 'AKTS', 'AKYA', 'ALDX', 'ALEC', 'ASTL', 'AQN', 'ALIT', 'ALHC', 'ALGS', 'ALLK', 'BIRD', 'ALLG', 'ALLO', 'ALLT', 'ALVR', 'ALUR', 'AAU', 'APT', 'ALPP', 
    'ATUS', 'ALT', 'ALTO', 'AMPS', 'ALXO', 'ALZN', 'AMRN', 'ABEV', 'AMCR', 'AXL', 'ACIC', 'AOUT', 'APEI', 'AREC', 'AMSC', 'AMWL', 'POWW', 'AMRX', 'AP', 'AMPY', 'AMPX', 'ASYS', 'AVXL', 'ANGI', 'ANGO', 'ANNX', 'HOUS', 'AIV', 
    'APPHQ', 'APLD', 'APDN', 'AMTI', 'APRE', 'APTO', 'AQMS', 'AQB', 'AQST', 'ARAV', 'ARBE', 'ABUS', 'ARC', 'RKDA', 'ACHR', 'FUV', 'ARQT', 'AMBP', 'ARDX', 'ACRE', 'ARBK', 'ARHS', 'ARKO', 'AAIC', 'ARLO', 'ARR', 'ARQQ', 'ARVL', 
    'AIP', 'ASX', 'ASXC', 'AHT', 'ASLN', 'ASPN', 'ASPU', 'ASMB', 'ASRT', 'ASTS', 'ASTR', 'ATXS', 'ATAI', 'ATRA', 'AVIR', 'ATER', 'ATHX', 'ATHA', 'ATIP', 'ATOM', 'ATOS', 'BCEL', 'LIFE', 'AUDA', 'AUPH', 'ACB', 'AUR', 'JG', 
    'AUTL', 'AVAH', 'AVPT', 'RNA', 'AWRE', 'AXGN', 'AXTI', 'AYRO', 'AZUL', 'BTG', 'BW', 'BLZE', 'BKKT', 'BLDP', 'BBAR', 'BBVA', 'BBD', 'BSBR', 'SAN', 'BFIN', 'BZUN', 'BCS', 'BBDC', 'BARK', 'BNED', 'BHC', 'BTE', 'BEEM', 
    'BLPH', 'BHIL', 'BEST', 'BTTX', 'XAIR', 'BYSI', 'BGC', 'BGFV', 'BIG', 'BBAI', 'BKYI', 'BPTH', 'BCAB', 'BCDA', 'BIOC', 'BCRX', 'BDSX', 'BFRI', 'BNGO', 'BIOR', 'BSGM', 'BTCY', 'BVS', 'BIVI', 'BTAI', 'BRDS', 'BTBT', 
    'BTCM', 'BITF', 'BNMV', 'BB', 'BKCC', 'BKSY', 'BLDE', 'BLND', 'BLNK', 'APRN', 'BLFY', 'BRBS', 'BLUE', 'BMTX', 'BORR', 'BOXDQ', 'BPT', 'BHR', 'BRAG', 'BCLI', 'BDN', 'BRCC', 'BRFS', 'BCTX', 'BGXX', 'BHG', 'BCOV', 'BRSP', 
    'BV', 'BRLT', 'BWEN', 'BKD', 'BRKL', 'BSQR', 'BFI', 'BFLY', 'BZFD', 'BYRN', 'CCCC', 'CDZI', 'CSTE', 'CAMP', 'CALC', 'CEI', 'CMBM', 'CAN', 'CANO', 'GOEV', 'CGC', 'CTLP', 'CFFN', 'CAPR', 'CGRN', 'CARA', 'CRDF', 'CRDL', 
    'CDNA', 'CMAX', 'CRBU', 'CARM', 'PRTS', 'TAST', 'CASA', 'CASI', 'CTRM', 'CATO', 'CZOO', 'CBAT', 'YCBD', 'CVM', 'CELC', 'CLBT', 'CLLS', 'CELU', 'CX', 'CENN', 'CNTA', 'CEPU', 'CENX', 'CNTY', 'IPSC', 'CRNT', 'CERS', 'CSBR', 
    'CRGE', 'CHPT', 'CLDT', 'CKPT', 'CMCM', 'CHGG', 'CSSE', 'CHS', 'CIM', 'CMRX', 'CAAS', 'CD', 'CDXC', 'CINT', 'CIG', 'CDTX', 'CIFR', 'CISO', 'CTXR', 'CIA', 'CIO', 'CLVT', 'CLAR', 'CLNE', 'CLSK', 'CCO', 'CLPT', 'CLSD', 
    'CLIR', 'CLNN', 'CLVR', 'CLPR', 'RAASY', 'CLOV', 'CODX', 'CDXS', 'CDE', 'CGNT', 'CHRS', 'CLGN', 'CVGI', 'COMM', 'CYH', 'CBD', 'ELP', 'SID', 'BVN', 'COMP', 'CMPS', 'CGEN', 'SCOR', 'LODE', 'CMTL', 'CNDT', 'CNTB', 'CONN', 
    'CNSL', 'CPSS', 'TCS', 'WISH', 'VLRS', 'CORR', 'CRMD', 'CRVS', 'COSM', 'CPSH', 'CPG', 'CRESY', 'CRCT', 'CRON', 'CRWS', 'LAW', 'CCLP', 'CUE', 'HLTH', 'CULP', 'CPIX', 'CMLS', 'CVAC', 'CURI', 'CRIS', 'CURO', 'CWK', 'CTOS', 
    'CUTR', 'CYCN', 'CYN', 'CTKB', 'CTSO', 'DADA', 'DAKT', 'DNMR', 'DRIO', 'DSKE', 'DAIO', 'DTEAF', 'DBVT', 'DBTX', 'DH', 'DCTH', 'DNN', 'DENN', 'DMTK', 'DSGN', 'DBI', 'DM', 'DESP', 'DXLG', 'DHX', 'DHT', 'DMAC', 'DRH', 'DSX', 
    'DGLY', 'APPS', 'DISH', 'DHC', 'DCGO', 'DOGZ', 'DLPN', 'DOMA', 'DOUG', 'DOYU', 'DPRO', 'DSHK', 'DRRX', 'DYAI', 'DLNG', 'DZSI', 'ETWO', 'KODK', 'EBON', 'EBET', 'ECVT', 'EDAP', 'EGIO', 'EDIT', 'EFTR', 'EGAN', 'EHTH', 
    'EIGR', 'EKSO', 'LOCO', 'EGO', 'SOLO', 'ELDN', 'EARN', 'ELYS', 'EMAN', 'EMKR', 'EEX', 'SOL', 'EBS', 'ESRT', 'EMX', 'EXK', 'WATT', 'UUUU', 'NRGV', 'ENFN', 'ENG', 'ENSV', 'ENTX', 'EGLX', 'EVC', 'ELA', 'NVRI', 'EVTV', 
    'EVA', 'NVNO', 'ENZ', 'EOSE', 'EPSN', 'EQOSQ', 'EQRX', 'EQ', 'EQX', 'ETRN', 'ERAS', 'ERIC', 'ESPR', 'GWH', 'EPIX', 'ERNA', 'EVAX', 'EVEX', 'EVLO', 'EVER', 'MRAM', 'EVGO', 'EVGN', 'EOLS', 'EPM', 'EVLV', 'XELA', 'EXFY', 
    'EXPR', 'EXAI', 'EYEN', 'EZPW', 'FANH', 'FTCH', 'FATE', 'FATH', 'FTHM', 'FAZE', 'FENC', 'GSM', 'FGH', 'FGEN', 'FRGI', 'FIGS', 'FOA', 'FNCH', 'FNGR', 'FINV', 'FFWM', 'AG', 'FRCB', 'SVVC', 'NOTE', 'FSR', 'FTK', 
    'FLNT', 'FLUX', 'FCUV', 'FHTX', 'FRSX', 'FRGE', 'FBRX', 'FBIO', 'FSM', 'FOSL', 'FSP', 'RAIL', 'FREQ', 'FREY', 'ULCC', 'HUGE', 'FTCI', 'FUBO', 'FTEK', 'FCEL', 'FULC', 'FLL', 'YMM', 'FNKO', 'HTOO', 'FUSN', 'FTFT', 'FF', 
    'GTHX', 'GAIA', 'GANX', 'GALT', 'GLTO', 'GRTX', 'GAU', 'GAME', 'GMDA', 'GAN', 'GCI', 'GOTU', 'GTX', 'GATO', 'GCMG', 'GNSS', 'WGS', 'GBIO', 'GEL', 'GENE', 'GNS', 'GENI', 'GNPX', 'GNW', 'GEO', 'GPRK', 'GEOS', 'GGB', 
    'GERN', 'GETY', 'GEVO', 'GILT', 'DNA', 'GLT', 'VRAR', 'GB', 'GBTG', 'GMRE', 'GSAT', 'GLBS', 'GLYC', 'GGR', 'GOL', 'GORO', 'GROY', 'GMGI', 'GOGL', 'GDRX', 'GPRO', 'GOSS', 'GRAB', 'EAF', 'GTE', 'GPMT', 'GRPH', 'GTN', 
    'AJX', 'GLDD', 'GREE', 'GP', 'GRFS', 'GRND', 'GRTS', 'GROV', 'GRWG', 'SUPV', 'TV', 'GSIT', 'GTBP', 'GIFI', 'HGTY', 'HOFV', 'HBI', 'HONE', 'HMY', 'HBIO', 'HE', 'HA', 'HL', 'HLGN', 'HLX', 'MOMO', 'HEPA', 'HRTG', 'HRTX', 
    'HT', 'HFFG', 'HITI', 'HLMN', 'HIMX', 'HIMS', 'HIPO', 'HIVE', 'HLLY', 'HMST', 'FIXX', 'HNST', 'HOOK', 'HUSA', 'HBM', 'HPP', 'HUMA', 'HUT', 'HUYA', 'HYMC', 'HYFM', 'HYLN', 'HYPR', 'HYREQ', 'HYZN', 'IMAB', 'IAG', 'IBIO', 
    'ICAD', 'ICL', 'ICLK', 'ICVX', 'IDR', 'IDEX', 'INVE', 'IGC', 'IGMS', 'IHRT', 'IHS', 'IMBIQ', 'IMMR', 'IMUX', 'IBRX', 'IMNM', 'IPA', 'IMMP', 'IMPP', 'ICD', 'INDI', 'ILPT', 'INFN', 'INFI', 'INMB', 'INNV', 'VATE', 'INVZ', 
    'INGN', 'NOTV', 'INO', 'INZY', 'INSG', 'ISPO', 'NSPR', 'TIL', 'IDN', 'INCR', 'TILE', 'IVAC', 'INTZ', 'NVTA', 'IVVD', 'IOVA', 'IQ', 'IREN', 'IRNT', 'IRWD', 'IRS', 'ISUN', 'ITP', 'ITUB', 'IZEA', 'JAGX', 'JSPR', 'JBLU', 
    'JOAN', 'JOBY', 'JYNT', 'JMIA', 'KLTR', 'KNDI', 'TOON', 'KPTI', 'KYN', 'KRNY', 'KTCC', 'KZR', 'KC', 'KNTE', 'KGC', 'KIRK', 'KNOP', 'KOD', 'KOPN', 'KEP', 'KRMD', 'KOS', 'KRON', 'KRO', 'KBNT', 'KULR', 'LE', 'LSEA', 'LNZA', 
    'LGO', 'LRMR', 'LTCH', 'SWIM', 'LFLY', 'LPTX', 'LC', 'LSAK', 'LESL', 'LXRX', 'LX', 'LPL', 'LICY', 'LILAK', 'LTRPA', 'LFCR', 'LFMD', 'LFST', 'LFVN', 'LTBR', 'ZEV', 'LWLG', 'LILM', 'LINC', 'LCTX', 'LGHL', 'LPCN', 'YVRLF', 
    'LQDA', 'LVO', 'LPSN', 'LVOX', 'LIZI', 'LL', 'LYG', 'LMFA', 'LDI', 'LOCL', 'LGMK', 'LOMA', 'LOOP', 'RIDEQ', 'LTRY', 'LXU', 'LCID', 'LU', 'LVLU', 'LUMN', 'LAZR', 'LUNA', 'LXP', 'LYEL', 'MGNX', 'MGTA', 'MX', 'MGNI', 
    'MHLD', 'TUSK', 'MNTX', 'MNKD', 'MRIN', 'MRNS', 'MKFG', 'MQ', 'MMLP', 'MTLS', 'MTRX', 'MTTR', 'MXCT', 'MBI', 'MUX', 'MFIN', 'MDVL', 'MAX', 'MPW', 'MNOV', 'MDWD', 'MEIP', 'MGTX', 'MERC', 'MRSN', 'MESA', 'MESO', 'MMAT', 
    'MTA', 'MVST', 'MVIS', 'MIST', 'MDXG', 'MNDO', 'MNMD', 'MIND', 'MIR', 'MIRO', 'AVO', 'MCW', 'MG', 'MUFG', 'MIXT', 'MFG', 'MOGO', 'MTEM', 'MNTS', 'MRCC', 'MNTK', 'MPAA', 'MOXC', 'MRC', 'MULN', 'MPLN', 'MYMD', 'MYTE', 
    'NAAS', 'NNDM', 'NNOX', 'NSTG', 'NNVC', 'NCMI', 'NTCO', 'NHTC', 'NWG', 'NAUT', 'NLS', 'NAVB', 'NM', 'NVTS', 'NKTR', 'NMRD', 'NLTX', 'NEON', 'NRDS', 'NRDY', 'NTWK', 'STIM', 'NMTC', 'NFGC', 'NGD', 'NEWP', 'NEGG', 'NMRK', 
    'NR', 'NEXA', 'NXE', 'NXDT', 'NEXT', 'KIND', 'NGL', 'NHWK', 'NKLA', 'NINE', 'NIU', 'NKTX', 'NNBR', 'NOK', 'NMR', 'NDLS', 'NAT', 'NAK', 'NG', 'NVAX', 'NRXP', 'NU', 'BURU', 'NRIX', 'SMR', 'NUVB', 'NVVE', 'NYMXF', 'OTLY', 
    'OBE', 'OCEA', 'OPTT', 'OCGN', 'OCUL', 'OCUP', 'OMEX', 'OPI', 'OIS', 'OLPX', 'OLMA', 'OLO', 'OMGA', 'OMER', 'ONTF', 'OCX', 'ONTX', 'ONCT', 'ONDS', 'OSS', 'OTRK', 'OPBK', 'OPAL', 'LPRO', 'OPEN', 'OPK', 'OPRT', 'OPFI', 
    'OPRX', 'ORMP', 'OSUR', 'OIG', 'ORC', 'OGI', 'ORGO', 'ONVO', 'ORGS', 'ORIC', 'OCG', 'ORGN', 'OESX', 'ORN', 'ONL', 'ORLA', 'OSCR', 'OTMO', 'OUST', 'OB', 'OSG', 'OVID', 'OWLT', 'OXSQ', 'PACB', 'PTVE', 'PACW', 'PGY', 
    'PAGS', 'PTN', 'PANL', 'PGRE', 'PRTK', 'PCYG', 'PASG', 'PAVM', 'PAYO', 'PAYS', 'PDSB', 'PTON', 'PNNT', 'PRET', 'PRSO', 'PFMT', 'PRM', 'PPIH', 'PVL', 'WOOF', 'PFSW', 'PMCB', 'PHXM', 'CELL', 'FENG', 'PHUN', 'PHX', 'PDM', 
    'PIRS', 'PBI', 'PXLW', 'PL', 'PLG', 'PLYA', 'AGS', 'MYPS', 'PLTK', 'PLBY', 'PLUG', 'PLUR', 'PMVP', 'PSNY', 'POL', 'PYPD', 'PRCH', 'PBPB', 'PWFL', 'PRAX', 'PGEN', 'PRPO', 'DTIL', 'PRLD', 'FRST', 'PROF', 'PROK', 'PGRU', 
    'PUMP', 'PRPH', 'PRQR', 'PSEC', 'PLX', 'TARA', 'PVBC', 'PLSE', 'PBYI', 'PCT', 'PPBT', 'PRPL', 'PYR', 'QUAD', 'QUBT', 'QSI', 'QS', 'QD', 'QRHC', 'QUIK', 'QNCX', 'QNST', 'QIPT', 'QUOT', 'QRTEA', 'QTTOY', 'RXT', 'RLGT', 
    'RFL', 'RAIN', 'METC', 'RANI', 'PACK', 'RAVE', 'RYAM', 'RDI', 'RCON', 'RXRX', 'RCAT', 'RDFN', 'RDW', 'RWT', 'REE', 'REED', 'RGLS', 'REKR', 'RLMD', 'MARK', 'RNLX', 'RNW', 'RENB', 'RENT', 'RPAY', 'FRBK', 'REFR', 'RSVR', 
    'RVP', 'RVPH', 'RWLK', 'RBBN', 'RIGL', 'RGTI', 'RMNI', 'REI', 'RSKD', 'RAD', 'RITM', 'RVSB', 'RLJ', 'RLX', 'RKLB', 'RMTI', 'ROVR', 'RES', 'RPT', 'RBT', 'RUM', 'RMBL', 'RSI', 'RVLP', 'RVYL', 'SANW', 'SABR', 'SACH', 
    'SGBX', 'SB', 'SFE', 'SBH', 'SJT', 'SANA', 'SAND', 'SGMO', 'SATL', 'SVRA', 'SRRK', 'SCLX', 'SCPH', 'SSP', 'SRL', 'SCYX', 'SEAC', 'SHIP', 'SCWX', 'SEEL', 'SEER', 'WTTR', 'SELB', 'SLQT', 'SLS', 'SEMR', 'SENS', 'SRTS', 
    'MCRB', 'SRG', 'SVC', 'SES', 'SHPW', 'SHCR', 'SFT', 'PIXY', 'SBSW', 'SIEB', 'SIEN', 'SIFY', 'SIGA', 'SGHT', 'SASI', 'SVM', 'SILV', 'SMWB', 'OMIC', 'SGLY', 'SIOX', 'SIRI', 'SKE', 'SKIL', 'SKLZ', 'SKYT', 'SND', 'SDC', 
    'SMSI', 'SNAP', 'SNPO', 'SNDL', 'SY', 'SCTL', 'SOFI', 'SHCO', 'SLGL', 'SLDP', 'DTC', 'SLNH', 'SLGC', 'SOND', 'SEVCQ', 'SOPH', 'SRNEQ', 'SOUN', 'SWN', 'LOV', 'ANY', 'SPI', 'SPIR', 'SBEV', 'SPWH', 'SPRU', 'SRAX', 'STGW', 
    'LAB', 'SLI', 'SRT', 'STKH', 'GASS', 'SCS', 'STEM', 'SFIX', 'STOK', 'SDIG', 'SMFG', 'INN', 'SMMT', 'SXC', 'SUNL', 'STKL', 'SPWR', 'SHO', 'SUNW', 'SGHC', 'SLGG', 'SUP', 'SURF', 'SURG', 'SSSS', 'STRO', 'SNCR', 'SYPR', 
    'SYRS', 'SST', 'TTOO', 'TBLA', 'TKAT', 'TAL', 'TLIS', 'TALK', 'TGB', 'TASK', 'TDCX', 'TK', 'TELA', 'TEO', 'VIV', 'TEF', 'TBIO', 'TELL', 'TLS', 'TIXT', 'TMPO', 'TNYA', 'TME', 'WULF', 'TERN', 'LLAP', 'TTI', 'TEVA', 'THCH', 
    'AREN', 'BODY', 'SKIN', 'LEV', 'RTL', 'STKS', 'REAX', 'REAL', 'NCTY', 'TXMD', 'THTX', 'TBPH', 'TWKS', 'TDUP', 'TTSH', 'TLYS', 'TLRY', 'TIO', 'TMC', 'CURV', 'TSQ', 'TRTX', 'TPIC', 'COOK', 'TACT', 'TAC', 'RIG', 'TG', 
    'TRMR', 'TRVN', 'TCN', 'TMQ', 'TRIB', 'DCFC', 'TGI', 'TRVG', 'TROO', 'TRUE', 'TRX', 'TOUR', 'TUP', 'TKC', 'TSP', 'TPC', 'TUYA', 'UCAR', 'GROW', 'USAU', 'UGP', 'UAA', 'UA', 'UFI', 'QURE', 'UIS', 'UMC', 'UAMY', 'UNIT', 
    'UEIC', 'UTI', 'TIGR', 'UPXI', 'UPH', 'UPLD', 'UEC', 'UROY', 'UONE', 'UWMC', 'UXIN', 'EGY', 'VCSA', 'VCNX', 'VLN', 'VLY', 'VNDA', 'VAPO', 'VBLT', 'VXRT', 'VBIV', 'VLD', 'VNTRQ', 'VRA', 'VERI', 'VRCA', 'VTNR', 'EVTL', 
    'VERU', 'VIA', 'DSP', 'RBOT', 'VRAYQ', 'VFF', 'VMEO', 'VWE', 'VQS', 'VIRX', 'SPCE', 'VHC', 'VTSI', 'VISL', 'VGZ', 'VTGN', 'VANI', 'SEAT', 'VVPR', 'VVOS', 'VZIO', 'VNET', 'VOC', 'VOD', 'VNRX', 'VOR', 'VOXX', 'VYGR', 
    'VRM', 'VTEX', 'VUZI', 'WTI', 'WBX', 'WVE', 'WEJOQ', 'HOWL', 'WPRT', 'WWR', 'WE', 'UP', 'WSR', 'FREE', 'WOW', 'WIMI', 'WIT', 'WISA', 'WT', 'WKEY', 'MAPS', 'WWW', 'WKHS', 'WKSP', 'WRAP', 'WW', 'XFOR', 'XBIT', 'XBIO', 
    'XERS', 'XOS', 'XNET', 'YMAB', 'YALA', 'YSG', 'YEXT', 'YRD', 'DAO', 'ZDGE', 'ZEPP', 'ZETA', 'ZVRA', 'ZH', 'ZUO', 'ZYME', 'ZYNE', 'ZYXI', 'AGNC', 'APT', 'AMSC', 'AVXL', 'APLD', 'ASTS', 'ACB', 'BHC', 'BBAI', 'BB', 'BLNK', 
    'CANO', 'GOEV', 'CGC', 'CHPT', 'CLOV', 'CODX', 'WISH', 'APPS', 'DISH', 'EDIT', 'EOSE', 'EXPR', 'FTCH', 'FGEN', 'AG', 'FSR', 'FUBO', 'FCEL', 'GNS', 'GETY', 'GSAT', 'GPRO', 'GRWG', 'HBI', 'HE', 'HL', 'HUT', 'IMPP', 
    'INO', 'IQ', 'JBLU', 'JMIA', 'KGC', 'LCID', 'LUMN', 'LAZR', 'MNKD', 'MPW', 'MMAT', 'MVIS', 'MULN', 'NNDM', 'NNOX', 'NEGG', 'NKLA', 'NOK', 'NVAX', 'NU', 'OPEN', 'PACW', 'PTON', 'PHUN', 'PLUG', 'PSNY', 'QS', 'RDFN', 'RDW', 
    'RKLB', 'RUM', 'SIRI', 'SKLZ', 'SDC', 'SNAP', 'SNDL', 'SOFI', 'SOUN', 'SWN', 'STEM', 'SFIX', 'SMMT', 'SPWR', 'TAL', 'TELL', 'TEVA', 'TLRY', 'RIG', 'TUP', 'UAA', 'UWMC', 'EVTL', 'VERU', 'SPCE', 'VOD', 'VRM', 'WE', 'WKHS', 
    'WW']

    # Remove duplicates
    unique_etfs = list(set(etfs))
    return sorted(unique_etfs)

In [3]:
def black_scholes_greeks(S, X, T, r, sigma, option_type='put'):
    d1 = (np.log(S / X) + (r + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    
    N_d1_prime = norm.pdf(d1)
    
    if option_type == 'call':
        C = S * N_d1 - X * np.exp(-r * T) * N_d2
        Delta = N_d1
        Gamma = N_d1_prime / (S * sigma * np.sqrt(T))
        Theta = -(S * N_d1_prime * sigma / (2 * np.sqrt(T))) - r * X * np.exp(-r * T) * N_d2
        Vega = S * np.sqrt(T) * N_d1_prime
        Rho = X * T * np.exp(-r * T) * N_d2
    elif option_type == 'put':
        C = X * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        Delta = N_d1 - 1
        Gamma = N_d1_prime / (S * sigma * np.sqrt(T))
        Theta = -(S * N_d1_prime * sigma / (2 * np.sqrt(T))) + r * X * np.exp(-r * T) * norm.cdf(-d2)
        Vega = S * np.sqrt(T) * N_d1_prime
        Rho = -X * T * np.exp(-r * T) * norm.cdf(-d2)
    else:
        raise ValueError("Invalid option type. Use either 'call' or 'put'.")
        
    return [C, Delta, Gamma, Theta, Vega, Rho]

In [4]:
def fetch_data(ticker, expiration_date):
    try:
        opts = yf.Ticker(ticker).option_chain(expiration_date)
        puts = opts.puts

        if 'bid' not in puts.columns:
            print(f'No options data for {ticker} on {expiration_date}')
            return pd.DataFrame()

        stock_price = round(yf.Ticker(ticker).info['currentPrice'], 2)
        condition = (puts['strike'] < stock_price) & (puts['bid'] > 0)
        puts = puts[condition].copy()
        stock_info = yf.Ticker(ticker).info

        # Add the ETF ticker to the DataFrame
        puts['ETF'] = ticker

        # Add the Stock Price to the DataFrame
        puts['Stock Price'] = stock_price
        # Add how out of the money the put is
        puts['OTM (%)'] = (((puts['Stock Price'] - puts['strike']) / puts['Stock Price']) * 100).round(2)
        # Add the expiration date to the DataFrame
        puts['Expiration Date'] = expiration_date
        # Add 'ROI (%)' column to DataFrame
        puts['ROI (%)'] = ((puts['bid'] / (puts['strike'] - puts['bid'])) * 100).round(2)
        # Add 'highPrice' column to DataFrame
        puts['highPrice'] = puts['bid']
        # Add 'Original ROI (%)' column to DataFrame
        puts['Original ROI (%)'] = ((puts['bid'] / (puts['strike'] - puts['bid'])) * 100).round(2)
        # Add 'Original OTM (%)' column to DataFrame
        puts['Original OTM (%)'] = (((puts['Stock Price'] - puts['strike']) / puts['Stock Price']) * 100).round(2)
        # Add 'Implied Volatility' column to DataFrame
        puts['Implied Volatility'] = puts['impliedVolatility'].round(2)
        # Add 'Original Implied Volatility' column to DataFrame
        puts['Original Implied Volatility'] = puts['impliedVolatility'].round(2)
        # Add 'Original Stock Price' column to DataFrame
        puts['Original Stock Price'] = stock_price
        # Add 'targetHighPrice' column to DataFrame
        puts['targetHighPrice'] = stock_info['targetHighPrice']
        # Add targetLowPrice' column to DataFrame
        puts['targetLowPrice'] = stock_info['targetLowPrice']
        # Add 'targetMeanPrice' column to DataFrame
        puts['targetMeanPrice'] = stock_info['targetMeanPrice']
        # Add 'targetMedianPrice' column to DataFrame
        puts['targetMedianPrice'] = stock_info['targetMedianPrice']
        # Add 'recommendationMean' column to DataFrame
        puts['recommendationMean'] = stock_info['recommendationMean']
        # Add 'recommendationKey' column to DataFrame
        puts['recommendationKey'] = stock_info['recommendationKey']

        # Calculate the number of days until expiration
        today = datetime.now()
        puts['daysToExpiration'] = (pd.to_datetime(puts['Expiration Date']) - today).dt.days

        # Calculate the Greeks
        greeks = black_scholes_greeks(puts['Stock Price'], puts['strike'], puts['daysToExpiration'] / 365, 0.01, puts['impliedVolatility'], option_type='put')
        # Add 'Delta' column to DataFrame
        puts['Delta'] = greeks[1].round(2)
        # Add 'Gamma' column to DataFrame
        puts['Gamma'] = greeks[2].round(2)
        # Add 'Theta' column to DataFrame
        puts['Theta'] = greeks[3].round(2)
        # Add 'Vega' column to DataFrame
        puts['Vega'] = greeks[4].round(2)
        # Add 'Rho' column to DataFrame
        puts['Rho'] = greeks[5].round(2)

        # Fetch historical data for the ETF ticker
        historical_data = yf.Ticker(ticker).history(period="1y")

        # Calculate current moving averages
        current_50_day_MA = historical_data['Close'].rolling(window=50).mean().iloc[-1]
        current_100_day_MA = historical_data['Close'].rolling(window=100).mean().iloc[-1]
        current_200_day_MA = historical_data['Close'].rolling(window=200).mean().iloc[-1]

        # Calculate current RSI
        delta = historical_data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean().iloc[-1]
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean().iloc[-1]
        current_RSI = 100 - (100 / (1 + (gain / loss)))

        # Calculate current MACD
        short_ema = historical_data['Close'].ewm(span=12, adjust=False).mean().iloc[-1]
        long_ema = historical_data['Close'].ewm(span=26, adjust=False).mean().iloc[-1]
        current_MACD = short_ema - long_ema

        # Calculate current VWAP
        typical_price = (historical_data['High'] + historical_data['Low'] + historical_data['Close']) / 3
        current_VWAP = np.cumsum(typical_price * historical_data['Volume']).iloc[-1] / np.cumsum(historical_data['Volume']).iloc[-1]

        # Add single value indicators to puts DataFrame
        puts['50-day MA'] = current_50_day_MA.round(2)
        puts['100-day MA'] = current_100_day_MA.round(2)
        puts['200-day MA'] = current_200_day_MA.round(2)
        puts['RSI'] = current_RSI.round(2)
        puts['MACD'] = current_MACD.round(2)
        puts['VWAP'] = current_VWAP.round(2)
        
        # Calculate 52-Week High and 52-Week Low
        high_52_week = historical_data['Close'].max()
        low_52_week = historical_data['Close'].min()

        puts['52WeekHigh'] = high_52_week.round(2)
        puts['52WeekLow'] = low_52_week.round(2)

        # Calculate the Break Even Price
        puts['Break Even Price'] = (puts['strike'] - puts['bid']).round(2)

        return puts
    except Exception as e:
        print(f"An error occurred: {e}")
        return

In [5]:
def screener(expiration_date):
    # Fetch all ETF tickers from the scraping function
    tickers = manual_etf_tickers()

    results = []

    for ticker in tickers:
        puts = fetch_data(ticker, expiration_date)
        if puts is not None:
            results.append(puts)

    # Check if results is empty
    if results is None:
        return pd.DataFrame()  # Return an empty dataframe

    if len(results) == 0:
        return pd.DataFrame()
    
    all_puts = pd.concat(results)

    return all_puts

In [6]:
def get_batch_ticker_prices(tickers, date):
    # Define Eastern Time Zone
    eastern = pytz.timezone('US/Eastern')

    # Convert date to 'YYYY-MM-DD' format string if it's a datetime.date object
    if isinstance(date, dt_date):  # Using the alias dt_date here
        date = date.strftime('%Y-%m-%d')

    # Convert target_date to Eastern Time Zone
    target_date = eastern.localize(datetime.strptime(date, '%Y-%m-%d'))  # Using datetime class here

    if not is_market_closed(target_date):
        print("The market has not closed yet for this date.")
        return None
    
    # Convert the end date to the next day
    end_date = (target_date + timedelta(days=1)).strftime('%Y-%m-%d')

    try:
        # Fetch data
        data = yf.download(tickers, start=date, end=end_date)
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return None
    
    if data.empty:
        print("No data found.")
        return None

    return data['Close']

In [7]:
def is_market_closed(target_date):
    market_close_time = time(16, 0)
    
    # Define Eastern Time Zone
    eastern = pytz.timezone('US/Eastern')
    
    # Check if target_date is already timezone-aware
    if target_date.tzinfo is None or target_date.tzinfo.utcoffset(target_date) is None:
        # Convert target_date to Eastern Time Zone
        target_date = eastern.localize(target_date)
    else:
        # Convert to Eastern Time if it's in a different timezone
        target_date = target_date.astimezone(eastern)
    
    # Get the current time in Eastern Time Zone
    current_time = datetime.now(eastern).time()

    # Check if the market has closed for the target date
    if target_date.date() < datetime.now(eastern).date() or \
       (target_date.date() == datetime.now(eastern).date() and current_time >= market_close_time):
        return True
    else:
        return False

In [8]:
def next_four_fridays():
    # Get today's date and time in EST timezone
    est = pytz.timezone('US/Eastern')
    today = datetime.now(est)
    
    # Find the next Friday
    days_until_friday = (4 - today.weekday() + 7) % 7
    next_friday = today + timedelta(days=days_until_friday)
    
    # Reset the time to 4:30 PM on the next Friday
    next_friday = next_friday.replace(hour=16, minute=30, second=0, microsecond=0)
    
    # If today is Friday and it's past 4:30 PM, consider the next week's Friday
    if today.weekday() == 4 and today.time() > next_friday.time():
        next_friday += timedelta(days=7)
    
    # Remove time from datetime for output
    next_friday = next_friday.date()
    
    # Find the Friday after the next
    following_friday = next_friday + timedelta(days=7)
    
    # Find the Friday after the following
    next_next_friday = following_friday + timedelta(days=7)
    
    # Find the Friday after the next next
    next_next_next_friday = next_next_friday + timedelta(days=7)
    
    return next_friday.strftime('%Y-%m-%d'), following_friday.strftime('%Y-%m-%d'), next_next_friday.strftime('%Y-%m-%d'), next_next_next_friday.strftime('%Y-%m-%d')

# Get the next four Fridays
friday1, friday2, friday3, friday4 = next_four_fridays()
print(f"The next four Fridays are: {friday1}, {friday2}, {friday3}, and {friday4}")

The next four Fridays are: 2023-09-08, 2023-09-15, 2023-09-22, and 2023-09-29


### Select the fridays you want from the next four fridays

In [13]:
fridays = [friday1]

### Looking into through Fridays ahead

In [10]:
desired_columns = ['contractSymbol','Expiration Date', 'ETF', 'Stock Price', 'lastPrice', 'bid', 'ask', 'strike', 'ROI (%)', 'OTM (%)', 'volume', 'openInterest', 'Implied Volatility', 'highPrice', 'Original Stock Price','Original ROI (%)', 
    'Original OTM (%)', 'Original Implied Volatility', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', '52WeekHigh', '52WeekLow', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice',
    'targetMedianPrice', 'recommendationMean', 'recommendationKey', 'numberOfAnalystOpinions', 'Break Even Price']
# Initialize results as an empty DataFrame with columns
results = pd.DataFrame(columns=desired_columns)

# Loop through each Friday
for x in fridays:
    # Get the screener DataFrame for each Friday
    screener_df = screener(x)
    
    # Check if the DataFrame is empty
    if screener_df.empty:
        print(f"No data found for {x}")
        continue
    
    # Append the screener DataFrame to results
    results = pd.concat([results, screener_df], ignore_index=True)

# List of columns to display
results = results[desired_columns]

# Make results a DataFrame
results = pd.DataFrame(results)

An error occurred: Expiration `2023-09-08` cannot be found. Available expirations are: [2023-09-15, 2023-10-20, 2024-01-19]
An error occurred: Expiration `2023-09-08` cannot be found. Available expirations are: [2023-09-15, 2023-12-15, 2024-03-15]
An error occurred: Expiration `2023-09-08` cannot be found. Available expirations are: [2023-09-15, 2023-10-20, 2024-01-19, 2024-04-19, 2025-01-17]
An error occurred: Expiration `2023-09-08` cannot be found. Available expirations are: [2023-09-15, 2023-10-20, 2023-12-15, 2024-03-15]
An error occurred: Expiration `2023-09-08` cannot be found. Available expirations are: [2023-09-15, 2023-10-20, 2024-01-19, 2025-01-17]
An error occurred: Expiration `2023-09-08` cannot be found. Available expirations are: [2023-09-15, 2023-10-20, 2024-01-19, 2024-04-19]
An error occurred: Expiration `2023-09-08` cannot be found. Available expirations are: [2023-09-15, 2023-10-20, 2023-11-17, 2024-02-16]
An error occurred: Expiration `2023-09-08` cannot be found. 

### Saving the data into CSV files

In [11]:
file_path = 'look_ahead.csv'

results.to_csv(file_path, index=False)
print("The 'look_ahead.csv' file has been updated successfully.")

The 'look_ahead.csv' file has been updated successfully.


In [12]:
file_viewer = pd.read_csv('look_ahead.csv')
filtered_data = file_viewer[file_viewer['Expiration Date'] == friday1]

# Display all rows, no limit
pd.set_option('display.max_rows', None)
# Display all columns, no limit
pd.set_option('display.max_columns', None)

filtered_data['ROI (%)'] = filtered_data['ROI (%)'].astype(float)
# filter out rows with ROI (%) greater than 0.8
# filtered_data = filtered_data[filtered_data['ROI (%)'] > 0.8]
# filter out rows with strike price less than 6
filtered_data = filtered_data[filtered_data['strike'] <= 9]

# sort by ROI (%) in ascending order
filtered_data = filtered_data.sort_values(by=['ROI (%)'], ascending=False)

# Show only the columns 'Expiration Date', 'ETF', 'Stock Price', 'bid', 'strike', 'ROI (%)', 'OTM (%)', 'Implied Volatility', 'volume', 'openInterest', 'highPrice', 'Original Stock Price', 'Original ROI (%)', 'Original OTM (%)', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', '52WeekHigh', and '52WeekLow'
filtered_data = filtered_data[['Expiration Date', 'ETF', 'Stock Price', 'bid', 'ask', 'strike', 'ROI (%)', 'OTM (%)', 'Implied Volatility', 'volume', 'Break Even Price', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice',
'targetMedianPrice', 'recommendationMean', 'recommendationKey', 'openInterest', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', '52WeekHigh', '52WeekLow']]

filtered_data

,Expiration Date,ETF,Stock Price,bid,ask,strike,ROI (%),OTM (%),Implied Volatility,volume,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,recommendationMean,recommendationKey,openInterest,50-day MA,100-day MA,200-day MA,RSI,MACD,VWAP,52WeekHigh,52WeekLow
18,2023-09-08,CGC,0.55,0.03,0.04,0.5,6.38,9.09,2.13,4595.0,3.29,0.42,1.44,1.02,3.8,underperform,3721.0,0.45,0.75,1.66,69.00,0.01,1.60,4.32,0.37
27,2023-09-08,EOSE,3.85,0.20,0.25,3.5,6.06,9.09,1.99,1243.0,15.00,4.00,5.92,4.00,2.4,buy,1931.0,3.35,2.87,2.23,73.33,0.16,2.64,5.03,0.97
23,2023-09-08,CHPT,7.44,0.32,0.33,7.0,4.79,5.91,1.40,519.0,27.00,10.00,14.99,14.00,1.9,buy,2396.0,7.99,8.37,9.59,41.55,-0.26,10.67,19.25,6.79
34,2023-09-08,GOEV,0.59,0.02,0.03,0.5,4.17,15.25,2.13,2613.0,10.00,0.50,3.11,1.55,2.0,buy,1539.0,0.56,0.60,0.80,59.28,0.01,0.80,2.89,0.43
57,2023-09-08,MVIS,2.51,0.10,0.11,2.5,4.17,0.40,0.86,599.0,4.00,4.00,4.00,4.00,3.0,hold,312.0,3.47,3.51,3.06,30.11,-0.29,3.93,7.65,1.82
66,2023-09-08,NVAX,8.21,0.25,0.32,8.0,3.23,2.56,0.92,314.0,38.00,5.00,20.60,15.00,2.7,hold,758.0,7.97,7.95,9.50,60.34,0.10,11.50,33.00,5.76
58,2023-09-08,NKLA,1.18,0.03,0.04,1.0,3.09,15.25,1.81,1010.0,5.00,1.00,2.42,2.25,2.9,hold,33575.0,1.85,1.35,1.78,26.30,-0.24,1.92,5.42,0.54
33,2023-09-08,FUBO,2.65,0.07,0.09,2.5,2.88,5.66,1.08,4634.0,5.00,3.00,3.60,3.00,2.7,hold,335.0,2.48,2.04,2.04,59.13,-0.05,2.45,5.12,1.04
17,2023-09-08,BLNK,4.01,0.11,0.15,4.0,2.83,0.25,0.66,38.0,50.00,7.00,16.89,10.00,2.7,hold,107.0,5.70,6.33,8.71,11.21,-0.58,9.30,23.23,3.94
22,2023-09-08,CHPT,7.44,0.17,0.18,6.5,2.69,12.63,1.45,2286.0,27.00,10.00,14.99,14.00,1.9,buy,1033.0,7.99,8.37,9.59,41.55,-0.26,10.67,19.25,6.79
